In [1]:
import numpy as np
import scipy.stats as stat

class WeightedMultinomialLogisticRegression(object):
    
    def __init__(self):
        """
        Constructor for the homgrown Logistic Regression
        
        Args:
            None
        
        Return:
            None
        """
        self.coef_ = None               # weight vector
        self.intercept_ = None          # bias term
        self._theta = None              # augmented weight vector, i.e., bias + weights
                                        # this allows to treat all decision variables homogeneously
        self.likelihood = None          # -2*log(L)
        self.AIC = None                 # AIC
        self.BIC = None                 # BIC
        self.Rsquare = None             # deviance R square
        self.adj_Rsquare = None         # adjusted deviance R square
        self.z_scores = {}              # z-score for theta
        self.p_values = {}              # p_values for theta
        self.fisher_inf_matrix = {}     # fisher information matrix
        self.sigma_estimates = {}       # sqaure root of covariance matrix for theta
        self.lower = {}                 # lower confidence interval for theta
        self.upper = {}                 # upper confidence interval for theta
        self.history = {"cost": [], 
                        "acc": [], 
                        "val_cost":[], 
                        "val_acc": []}
        
    def _grad(self, X, y, weight):
        """
        Calculates the gradient of the Logistic Regression 
        objective function

        Args:
            X(ndarray):      train objects
            y(ndarray):      answers for train objects
            weight(ndarray): weights for train objects for weighted logistic regression
            
        Return:
            grad(ndarray): gradient
        """
        # number of training examples
        n = X.shape[0]
        
        # get scores for each class and example
        # 2D matrix
        scores = self._predict_raw(X)
        
        # transform scores to probabilities
        # softmax
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        
        # error
        probs[range(n),y] -= 1
        
        # gradient
        gradient = np.dot(X.T, weight.reshape(n, 1)*probs) / n
        
        return gradient
    
    def _gd(self, X, y, weight, max_iter, eta, X_val, y_val):
        """
        Runs Full GD and logs error, weigths, gradient at every step

        Args:
            X(ndarray):      train objects
            y(ndarray):      answers for train objects
            weight(ndarray): weights for train objects for weighted logistic regression
            max_iter(int):   number of weight updates
            eta(float):      step size in direction of gradient
            
        Return:
            None
        """
        for i in range(max_iter):
            
            metrics = self.score(X, y)
            self.history["cost"].append(metrics["cost"])
            self.history["acc"].append(metrics["acc"])
            
            if X_val is not None:
                metrics_val = self.score(X_val, y_val)
                self.history["val_cost"].append(metrics_val["cost"])
                self.history["val_acc"].append(metrics_val["acc"])

            # calculate gradient
            grad = self._grad(X, y, weight)
            
            # do gradient step
            self._theta -= eta * grad
    
    def fit(self, X, y, weight=None, max_iter=5000, eta=0.01, val_data=None):
        """
        Public API to fit Logistic regression model
        
        Args:
            X(ndarray):      train objects
            y(ndarray):      answers for train objects
            weight(ndarray): wweights for train objects for weighted logistic regression
            max_iter(int):   number of weight updates
            alpha(float):    step size in direction of gradient
            
        Return:
            None
        """
        # Augment the data with the bias term.
        X = np.c_[np.ones(X.shape[0]), X]
        if val_data is not None:
            X_val, y_val = val_data
            X_val = np.c_[np.ones(X_val.shape[0]), X_val]
        else:
            X_val = None
            y_val = None
        # initialize if the first step
        if self._theta is None:
            self._theta = np.random.rand(X.shape[1], len(np.unique(y)))
        #initialize the weight
        if weight is None:
            weight = np.ones(X.shape[0]) 
        
        # do full gradient descent
        self._gd(X, y, weight, max_iter, eta, X_val, y_val)
        
        # get final weigths and bias
        self.intercept_ = self._theta[0, :]
        self.coef_ = self._theta[1:, :]
        
        #calculate the various statistics
        self._statistics_(X, y, weight)

    
    def _statistics_(self, X, y, w):
        """
        calculate the various statistics for the model
        
        Args:
            X(ndarray):      objects
            y(ndarray):      answers for objects
            w(ndarray):      weights for objects
            
        Return:
            None
        """
        
        #calculate the various model diagnostic statistics
        #the code is from 
        #https://gist.github.com/rspeare/77061e6e317896be29c6de9a85db301d
        n = X.shape[0]
        p = X.shape[1]
        
        for i in range(np.unique(y).shape[0]):
            denom = (2.0*(1.0+np.cosh(self.decision_function(X)[:, i])))
            denom = np.tile(denom,(p,1)).T
            fisher_inf_matrix = np.dot((X/denom).T,X*w[:, np.newaxis]) ## Fisher Information Matrix
            Cramer_Rao = np.linalg.inv(fisher_inf_matrix) ## Inverse Information Matrix
            sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
            
            z_scores = self._theta[:, i]/sigma_estimates # z-score for eaach model coefficient
            p_values = [stat.norm.sf(abs(x))*2 for x in z_scores] ### two tailed test for p-values

            alpha = 0.05
            q = stat.norm.ppf(1 - alpha / 2)
            lower = self._theta[:, i] - q * sigma_estimates
            upper = self._theta[:, i] + q * sigma_estimates

            self.z_scores[i] = z_scores
            self.p_values[i] = p_values
            self.sigma_estimates[i] = sigma_estimates
            self.fisher_inf_matrix[i] = fisher_inf_matrix
            self.lower[i] = lower                
            self.upper[i] = upper
        
        scores = self._predict_raw(X)
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        
        probs_null = np.bincount(y)/n
        probs_null = np.tile(probs_null, (n, 1))
        
        self.likelihood = np.sum(np.log(probs[range(n),y])*w)
        likelihood_null = np.sum(np.log(probs_null[range(n), y])*w)
        self.Rsquare = 1 - self.likelihood/likelihood_null
        self.adj_Rsquare = 1 - (self.likelihood/(n-p-1))/(likelihood_null/(n-1))
        
        self.AIC = 2*p - 2*self.likelihood
        self.BIC = p*np.log(n) - 2*self.likelihood
        
    def score(self, X, y):
        """
        Computes logloss and accuracy for (X, y)
        
        Args:
            X(ndarray):      objects
            y(ndarray):      answers for objects
            
        Return:
            metrics(dict):   python dictionary which
                             contains two fields: for accuracy 
                             and for objective function
        """
        # number of training samples
        n = X.shape[0]
        
        # get scores
        scores = self._predict_raw(X)
        
        # trasnform scores to probabilities
        exp_scores = np.exp(scores)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        
        # logloss per each example
        corect_logprobs = -np.log(probs[range(n),y])
        
        # total mean logloss
        data_loss = np.sum(corect_logprobs) / n
        
        # predictions
        pred = np.argmax(scores, axis=1)
        # accuracy
        acc = accuracy_score(y, pred)
        
        # final metrics
        metrics = {"acc": acc, "cost": data_loss}
        
        return metrics
    
    def _predict_raw(self, X):
        """
        ...... each class and each object in X
        calculate the raw score and substract the maximum to avoid overflow
        
        Args:
            X(ndarray):      objects
        
        Return:
            ...(ndarray): ...... each class and object
        """
        if X.shape[1] == len(self._theta):
            scores = np.dot(X, self._theta)
        else:
            scores = np.dot(X, self.coef_) + self.intercept_
        
        scores -= np.max(scores, axis=1, keepdims=True)
        
        return scores    
    
    def decision_function(self, X):
        """
        ...... each class and each object in X
        calculate the raw score without substracing the maximum
        
        Args:
            X(ndarray):      objects
        
        Return:
            ...(ndarray): ...... each class and object
        """
        if X.shape[1] == len(self._theta):
            scores_ = np.dot(X, self._theta)
        else:
            scores_ = np.dot(X, self.coef_) + self.intercept_
        
        return scores_
    
    def predict(self, X):
        """
        Predicts class for each object in X
        
        Args:
            X(ndarray):      objects
        
        Return:
            pred(ndarray):   class for each object
        """
        # get scores for each class
        scores = self._predict_raw(X)
        # choose class with maximum score
        pred = np.argmax(scores, axis=1)
        return pred

In [2]:
import numpy as np
import scipy.stats as stat
import pandas as pd

class WeightedBinaryLogisticRegression(object):
    
    def __init__(self):
        """
        Constructor for Weighted Binary Logistic Regression
        
        Args:
            None
        
        Return:
            None
        """
        self.coef_ = None               # weight vector
        self.intercept_ = None          # bias term
        self._theta = None              # augmented weight vector, i.e., bias + weights
                                        # this allows to treat all decision variables homogeneously
        self.likelihood = None          # -2*log(L)
        self.AIC = None                 # AIC
        self.BIC = None                 # BIC
        self.Rsquared = None            # deviance R squared
        self.adj_Rsquared = None        # adjusted deviance R squared
        self.z_scores = None            # z_scores for theta
        self.p_values = None            # p_values for theta
        self.fisher_inf_matrix = None   # fisher information matrix                              
        self.sigma_estimates = None     # sqaure root of covariance matrix for theta
        self.lower = None               # lower confidence interval for theta
        self.upper = None               # upper confidence interval for theta
        self.history = {"cost": [], 
                        "acc": [], 
                        "val_cost":[], 
                        "val_acc": []}
        
    def _grad(self, X, y, weight):
        """
        Calculates the gradient of the Logistic Regression 
        objective function

        Args:
            X(ndarray):      train objects
            y(ndarray):      answers for train objects
            weight(ndarray): weights for train objects for weighted logistic regression
            
        Return:
            grad(ndarray): gradient
        """
        # number of training examples
        n = X.shape[0]
        
        # get scores for each class and example
        scores = self._predict_raw(X)
        
        # transform scores to probabilities
        probs = np.zeros(X.shape[0])
        probs[(scores>=0).flatten()] = 1/(1+np.exp(-1*scores[scores>=0].flatten()))
        probs[(scores<0).flatten()] = 1 - 1/(1+np.exp(scores[scores<0].flatten()))
        
        # error
        probs[y == 1] -= 1
        
        # gradient
        gradient = np.dot(X.T, (weight*probs).reshape(n, 1)) / n
        
        return gradient
    
    def _gd(self, X, y, weight, max_iter, eta, X_val, y_val):
        """
        Runs Full GD and logs error, weigths, gradient at every step

        Args:
            X(ndarray):      train objects
            y(ndarray):      answers for train objects
            weight(ndarray): weights for train objects for weighted logistic regression
            max_iter(int):   number of weight updates
            eta(float):      step size in direction of gradient
            
        Return:
            None
        """
        for i in range(max_iter):
            
            metrics = self.score(X, y)
            self.history["cost"].append(metrics["cost"])
            self.history["acc"].append(metrics["acc"])
            
            if X_val is not None:
                metrics_val = self.score(X_val, y_val)
                self.history["val_cost"].append(metrics_val["cost"])
                self.history["val_acc"].append(metrics_val["acc"])

            # calculate gradient
            grad = self._grad(X, y, weight)
            
            # do gradient step
            self._theta -= eta * grad
    
    def fit(self, X, y, weight=None, max_iter=5000, eta=0.01, val_data=None):
        """
        Public API to fit Logistic regression model
        
        Args:
            X(ndarray):      train objects
            y(1darray):      answers for train objects
            weight(ndarray): wweights for train objects for weighted logistic regression
            max_iter(int):   number of weight updates
            alpha(float):    step size in direction of gradient
            
        Return:
            None
        """
        # Augment the data with the bias term.
        X = np.c_[np.ones(X.shape[0]), X]
        if val_data is not None:
            X_val, y_val = val_data
            X_val = np.c_[np.ones(X_val.shape[0]), X_val]
        else:
            X_val = None
            y_val = None
        # initialize if the first step
        if self._theta is None:
            self._theta = np.random.rand(X.shape[1], 1)
        #initialize the weight
        if weight is None:
            weight = np.ones(X.shape[0]) 
        
        # do full gradient descent
        self._gd(X, y, weight, max_iter, eta, X_val, y_val)
        
        # get final weigths and bias
        self.intercept_ = self._theta[0]
        self.coef_ = self._theta[1:]
        
        #calculate the various statistics
        self._statistics_(X, y, weight)

    
    def _statistics_(self, X, y, w):
        """
        calculate the various statistics for the model
        
        Args:
            X(ndarray):      objects
            y(ndarray):      answers for objects
            w(ndarray):      weights for objects
            
        Return:
            None
        """
        
        #calculate the various model diagnostic statistics
        #the code is from 
        #https://gist.github.com/rspeare/77061e6e317896be29c6de9a85db301d
        n = X.shape[0]
        p = X.shape[1]
        
        denom = (2.0*(1.0+np.cosh(self.decision_function(X)))).flatten()
        denom = np.tile(denom,(p, 1)).T
        fisher_inf_matrix = np.dot((X/denom).T, X*w[:, np.newaxis]) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(fisher_inf_matrix) ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))

        z_scores = self._theta.flatten()/sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x))*2 for x in z_scores] ### two tailed test for p-values

        alpha = 0.05
        q = stat.norm.ppf(1 - alpha / 2)
        lower = self._theta.flatten() - q * sigma_estimates
        upper = self._theta.flatten() + q * sigma_estimates

        self.z_scores = z_scores
        self.p_values = p_values
        self.sigma_estimates = sigma_estimates
        self.fisher_inf_matrix = fisher_inf_matrix
        self.lower = lower                
        self.upper = upper
        
        probs = self.predict_probs(X) 
        
        probs_null = np.bincount(y)/n
        probs_null = np.tile(probs_null, (n, 1))
        
        self.likelihood = np.sum(np.log(probs[range(n),y])*w)
        likelihood_null = np.sum(np.log(probs_null[range(n), y])*w)
        self.Rsquared = 1 - self.likelihood/likelihood_null
        self.adj_Rsquared = 1 - (self.likelihood/(n-p-1))/(likelihood_null/(n-1))
        
        self.AIC = 2*p - 2*self.likelihood
        self.BIC = p*np.log(n) - 2*self.likelihood
        
    def score(self, X, y):
        """
        Computes logloss and accuracy for (X, y)
        
        Args:
            X(ndarray):      objects
            y(ndarray):      answers for objects
            
        Return:
            metrics(dict):   python dictionary which
                             contains two fields: for accuracy 
                             and for objective function
        """
        # number of training samples
        n = X.shape[0]
        
        # get probabilities
        probs = self.predict_probs(X)
        
        # logloss 
        data_loss = np.sum(-np.log(probs[range(n),y]))/n
        
        # predictions
        pred = np.argmax(probs, axis=1)
        # accuracy
        acc = accuracy_score(y, pred)
        
        # final metrics
        metrics = {"acc": acc, "cost": data_loss}
        
        return metrics
    
    def _predict_raw(self, X):
        """
        ...... each class and each object in X
        calculate the raw score and substract the maximum to avoid overflow
        
        Args:
            X(ndarray):      objects
        
        Return:
            ...(ndarray): ...... each class and object
        """        
        if X.shape[1] == len(self._theta):
            scores = np.dot(X, self._theta)
        else:
            scores = np.dot(X, self.coef_) + self.intercept_
        
        return scores   
    
    def decision_function(self, X):
        """
        ...... each class and each object in X
        calculate the raw score without substracing the maximum
        
        Args:
            X(ndarray):      objects
        
        Return:
            ...(ndarray): ...... each class and object
        """
        return self._predict_raw(X)
    
    def predict_probs(self, X):
        """
        ...... each class and each object in X
        calculate the raw score without substracing the maximum
        
        Args:
            X(ndarray):      objects
        
        Return:
            ...(ndarray): ...... each class and object
        """
        scores = self._predict_raw(X)
        probs = np.zeros(X.shape[0])
        probs[(scores>=0).flatten()] = 1/(1+np.exp(-1*scores[scores>=0].flatten()))
        probs[(scores<0).flatten()] = 1 - 1/(1+np.exp(scores[scores<0].flatten()))
        probs = np.hstack(((1-probs).reshape(X.shape[0], 1), probs.reshape(X.shape[0], 1)))
        
        return probs
    
    def predict(self, X):
        """
        Predicts class for each object in X
        
        Args:
            X(ndarray):      objects
        
        Return:
            pred(ndarray):   class for each object
        """
        # get scores for each class
        probs = probs = self.predict_probs(X)
        # choose class with maximum score
        pred = np.argmax(probs, axis=1)
        return pred
    
    def get_statistics(self):
#         import pdb; pdb.set_trace()
        df_coef_stats = pd.DataFrame(np.zeros((X.shape[1]+1, 6)), columns=['estimate', 'sd err', 'z_score', 
                                                        'p_value', 'lower', 'upper'])
        df_model_stats = pd.DataFrame(np.zeros((1, 5)), columns=['logLikelihood', 'R_sqaured', 'adj_R_squared',
                                                              'AIC', "BIC"])
        
        X_name = ['intercept']
        for i in range(1, X.shape[1]+1):
            X_name.append('X'+str(i))
        df_coef_stats.index = X_name
        
        df_coef_stats['estimate'] = self._theta
        df_coef_stats['sd err'] = self.sigma_estimates
        df_coef_stats['z_score'] = self.z_scores
        df_coef_stats['p_value'] = self.p_values
        df_coef_stats['lower'] = self.lower
        df_coef_stats['upper'] = self.upper
        
        df_model_stats.iloc[0, 0] = self.likelihood
        df_model_stats.iloc[0, 1] = self.Rsquared
        df_model_stats.iloc[0, 2] = self.adj_Rsquared
        df_model_stats.iloc[0, 3] = self.AIC
        df_model_stats.iloc[0, 4] = self.BIC
        
        return (df_model_stats, df_coef_stats)

# Import the iris data

In [3]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from statsmodels.discrete.discrete_model import Logit
import statsmodels.api as sm
from sklearn.metrics import accuracy_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid", font_scale=1.3)
matplotlib.rcParams["legend.framealpha"] = 1
matplotlib.rcParams["legend.frameon"] = True

np.random.seed(42)

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Let's compare the homegrown method to the methods from sklearn and statsmodel**

# Binary case

0: non-versicolour and 1: versicolour

In [5]:
y_bi_train = (y_train == 1).astype(int)
y_bi_test = (y_test == 1).astype(int)

## sklearn

In [6]:
results = pd.DataFrame(columns=['intercept', 'X1', 'X2', 'X3', 'X4', 'train accuracy', 'test accuracy'])

models = {'sklearn Logistic Regression Ovr': LogisticRegression(C=1e6, multi_class="ovr", solver='liblinear'),
         'sklearn Logistic Regression Multinomial': LogisticRegression(C=1e6, multi_class="multinomial", solver='lbfgs'),
         'sklearn SGD': SGDClassifier(loss='log', penalty='l2', alpha=1e-6, max_iter=10000, tol=1e-6)}

for key, model in models.items():
    model.fit(X_train, y_bi_train)
    results = results.append(pd.DataFrame(
            {'intercept': model.intercept_, 'X1': model.coef_[0, 0], 
             'X2':model.coef_[0, 1], 'X3':model.coef_[0, 2], 'X4':model.coef_[0, 3], 
             'train accuracy': np.round(accuracy_score(y_bi_train, model.predict(X_train)), 3), 
             'test accuracy': np.round(accuracy_score(y_bi_test, model.predict(X_test)), 3)},
            index=[key]))


In [7]:
results

,X1,X2,X3,X4,intercept,test accuracy,train accuracy
sklearn Logistic Regression Ovr,0.048248,-2.906586,1.016315,-2.373328,6.691446,0.833,0.708
sklearn Logistic Regression Multinomial,0.023765,-1.453643,0.508242,-1.186591,3.348442,0.833,0.708
sklearn SGD,417.547623,-883.891077,280.851093,-739.327237,313.157542,0.633,0.692


## statsmodel

In [8]:
X_train_ = sm.add_constant(X_train)
X_test_ = sm.add_constant(X_test)
logit = Logit(y_bi_train, X_train_)
res = logit.fit(method='bfgs', maxiter=5000)
preds_train = res.predict(X_train_)
preds_train = (preds_train >= 0.5).astype(int)
preds_test = res.predict(X_test_)
preds_test = (preds_test >= 0.5).astype(int)

Optimization terminated successfully.
         Current function value: 0.492163
         Iterations: 37
         Function evaluations: 39
         Gradient evaluations: 39


In [9]:
res.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:                Logit            Pseudo R-squared: 0.234   
Dependent Variable:   y                AIC:              128.1190
Date:                 2019-03-08 21:09 BIC:              142.0565
No. Observations:     120              Log-Likelihood:   -59.060 
Df Model:             4                LL-Null:          -77.056 
Df Residuals:         115              Scale:            1.0000  
Converged:            1.0000                                     
-----------------------------------------------------------------
                   Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
-----------------------------------------------------------------
const              6.6969   2.6304  2.5460 0.0109  1.5414 11.8523
sepal length (cm)  0.0475   0.6998  0.0679 0.9458 -1.3240  1.4190
sepal width (cm)  -2.9073   0.8612 -3.3758 0.0007 -4.5952 -1.2194
petal length (cm)  1.0165   0.7509  1.3538 0.1758 -0.4551  2.4882
petal width (cm)  -2.3732   1.3002 -1.8254 0.0679 -4.9215  0.1750
=================================================================

"""

In [10]:
results = results.append(pd.DataFrame(
        {'intercept': res.params[0], 'X1': res.params[1], 
         'X2':res.params[2], 'X3':res.params[3], 'X4':res.params[4], 
         'train accuracy': np.round(accuracy_score(y_bi_train, preds_train), 3), 
         'test accuracy': np.round(accuracy_score(y_bi_test, preds_test), 3)},
        index=['statsmodel Logit']))

In [11]:
results

,X1,X2,X3,X4,intercept,test accuracy,train accuracy
sklearn Logistic Regression Ovr,0.048248,-2.906586,1.016315,-2.373328,6.691446,0.833,0.708
sklearn Logistic Regression Multinomial,0.023765,-1.453643,0.508242,-1.186591,3.348442,0.833,0.708
sklearn SGD,417.547623,-883.891077,280.851093,-739.327237,313.157542,0.633,0.692
statsmodel Logit,0.047534,-2.907298,1.016506,-2.373235,6.696884,0.833,0.708


## homegrown

In [12]:
model_wlr = WeightedBinaryLogisticRegression()
model_wlr.fit(X_train, y_bi_train, max_iter=50000, eta=0.1)

In [13]:
results = results.append(pd.DataFrame(
        {'intercept': model_wlr._theta[0], 'X1': model_wlr._theta[1], 
         'X2':model_wlr._theta[2], 'X3':model_wlr._theta[3], 'X4':model_wlr._theta[4], 
         'train accuracy': np.round(accuracy_score(y_bi_train, model_wlr.predict(X_train)), 3), 
         'test accuracy': np.round(accuracy_score(y_bi_test, model_wlr.predict(X_test)), 3)},
        index=['Homegrown Weighted Binary Logisitc']))

In [14]:
df1, df2 = model_wlr.get_statistics()

compare the model statistics from homegrown method to statsmodel

In [15]:
df1

,logLikelihood,R_sqaured,adj_R_squared,AIC,BIC
0,-59.059529,0.233552,0.199936,128.119057,142.056516


In [16]:
df2

,estimate,sd err,z_score,p_value,lower,upper
intercept,6.679428,2.629149,2.540529,0.011069,1.526391,11.832465
X1,0.049510,0.699633,0.070765,0.943585,-1.321746,1.420765
X2,-2.904532,0.860787,-3.374276,0.000740,-4.591642,-1.217421
X3,1.015781,0.750680,1.353148,0.176008,-0.455525,2.487086
X4,-2.372817,1.299877,-1.825417,0.067938,-4.920529,0.174894


compare the results from all models

In [17]:
results

,X1,X2,X3,X4,intercept,test accuracy,train accuracy
sklearn Logistic Regression Ovr,0.048248,-2.906586,1.016315,-2.373328,6.691446,0.833,0.708
sklearn Logistic Regression Multinomial,0.023765,-1.453643,0.508242,-1.186591,3.348442,0.833,0.708
sklearn SGD,417.547623,-883.891077,280.851093,-739.327237,313.157542,0.633,0.692
statsmodel Logit,0.047534,-2.907298,1.016506,-2.373235,6.696884,0.833,0.708
Homegrown Weighted Binary Logisitc,0.049510,-2.904532,1.015781,-2.372817,6.679428,0.833,0.708


# Multinomial Case

## sklearn

In [18]:
mn_model_sklearnLogistic = LogisticRegression(C=1e6, multi_class="multinomial", solver='lbfgs')
mn_model_sklearnLogistic.fit(X_train, y_train)

LogisticRegression(C=1000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='multinomial', n_jobs=None, penalty='l2',
          random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
          warm_start=False)

In [19]:
result_mn = []
result_mn.append(pd.DataFrame(np.hstack((mn_model_sklearnLogistic.intercept_.reshape((3, 1)), 
                                  mn_model_sklearnLogistic.coef_)).T, columns=['setosa', 'versicolour', 'virginica']))

In [20]:
result_acc = pd.DataFrame(columns=['train accuracy', 'test accuracy'])
result_acc = result_acc.append(pd.DataFrame({'train accuracy':accuracy_score(y_train, mn_model_sklearnLogistic.predict(X_train)),
                        'test accuracy':accuracy_score(y_test, mn_model_sklearnLogistic.predict(X_test)) }, 
                    index=['sklearn_logistic']))

In [21]:
mn_model_SGD = SGDClassifier(loss='log', penalty='l2', alpha=1e-6, max_iter=10000, tol=1e-6)
mn_model_SGD.fit(X_train, y_train)

SGDClassifier(alpha=1e-06, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=10000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=1e-06,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [22]:
result_mn.append(pd.DataFrame(np.hstack((mn_model_SGD.intercept_.reshape((3, 1)), 
                                  mn_model_SGD.coef_)).T, columns=['setosa', 'versicolour', 'virginica']))

In [23]:
result_acc = result_acc.append(pd.DataFrame({'train accuracy':accuracy_score(y_train, mn_model_SGD.predict(X_train)),
                        'test accuracy':accuracy_score(y_test, mn_model_SGD.predict(X_test))}, 
                    index=['sklearn_sgd']))

## Statsmodel

In [24]:
mnlogit = sm.MNLogit(y_train, X_train_)
mn_res = mnlogit.fit(method='bfgs', maxiter=5000)
preds_train_ = mn_res.predict(X_train_)
preds_train_ = np.argmax(preds_train_.values, axis=1)
preds_test_ = mn_res.predict(X_test_)
preds_test_ = np.argmax(preds_test_.values, axis=1)

Optimization terminated successfully.
         Current function value: 0.046578
         Iterations: 64
         Function evaluations: 66
         Gradient evaluations: 66


In [25]:
result_mn.append(pd.DataFrame(mn_res.params.values, columns=['setosa', 'versicolour']))

In [26]:
result_acc = result_acc.append(pd.DataFrame({'train accuracy':accuracy_score(y_train, preds_train_),
                        'test accuracy':accuracy_score(y_test, preds_test_)}, 
                    index=['statsmodel MNLogit']))

## homegrown 

In [33]:
# using raw gradient descent seems to  be really slow
# need to implement lbfgs or other algorithm for speed
%%time
mn_model_wlr = WeightedMultinomialLogisticRegression()
mn_model_wlr.fit(X_train, y_train, max_iter=100000, eta=0.1)

Wall time: 36 s


In [34]:
mn_model_wlr._theta

array([[ 1.91051048,  8.71272432, -8.99897806],
       [ 3.08943081,  0.86441483, -1.62730551],
       [ 6.07475898,  0.99602217, -5.19701774],
       [-6.58229067,  1.04687124,  7.38370817],
       [-3.44244263, -3.93823869,  7.85785818]])

In [35]:
result_mn.append(pd.DataFrame(mn_model_wlr._theta, columns=['setosa', 'versicolour', 'virginica']))

In [36]:
result_coef_mn = pd.concat(result_mn, keys=['sklearn_logistic', 'sklearn_sgd', 'statsmodel', 'homegrown'], axis=1)

In [37]:
result_acc = result_acc.append(pd.DataFrame({'train accuracy':accuracy_score(y_train, mn_model_wlr.predict(X_train)),
                        'test accuracy':accuracy_score(y_test, mn_model_wlr.predict(X_test))}, 
                    index=['homegrown']))

compare the fitted coefficients

In [38]:
result_coef_mn.index = ['intercept', 'X1', 'X2', 'X3', 'X4']
result_coef_mn

sklearn_logistic                        sklearn_sgd               \
                    setosa versicolour  virginica      setosa  versicolour   
intercept         2.206041   16.730197 -18.936238   15.808389   292.531497   
X1                4.322002   -1.094362  -3.227641   18.483277   486.386042   
X2                9.933641   -1.536074  -8.397567   81.634472 -1019.513567   
X3              -14.528462    3.109692  11.418770  -98.577476   348.470155   
X4               -6.881934   -4.788829  11.670762  -38.506827  -653.107398   

                       statsmodel             homegrown                        
             virginica     setosa versicolour    setosa versicolour virginica  
intercept  -925.212410  14.605506  -21.068110  1.910510    8.712724 -8.998978  
X1        -1167.397472  -5.278503   -7.411876  3.089431    0.864415 -1.627306  
X2        -1320.851814 -12.643246  -19.505523  6.074759    0.996022 -5.197018  
X3         1880.543948  18.475378   26.785759 -6.582291    1.046871  7.383708  
X4         1710.828316   2.141630   18.603335 -3.442443   -3.938239  7.857858

compare the accuracy

In [39]:
result_acc

,test accuracy,train accuracy
sklearn_logistic,1.000000,0.983333
sklearn_sgd,0.866667,0.850000
statsmodel MNLogit,1.000000,0.983333
homegrown,1.000000,0.983333
